In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [21]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [22]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [23]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [24]:
roles = {'state':['explanatory','groupby'], 'year':'trend', 'driver_gender':['explanatory','groupby'], 'driver_race':['explanatory','groupby'],
       'decriminalization':['explanatory','groupby'], 'medical':['explanatory','groupby'],
         'recreational':['explanatory','groupby'], 'no_reforms':['explanatory','groupby'],
       'search_conducted_false':'trend', 'search_conducted_true':'trend',
       'search_conducted_rate':'trend', 'contraband_found_false':'trend',
       'contraband_found_true':'trend', 'contraband_found_rate':'trend', 'hit_false':'trend',
       'hit_true':'trend', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [25]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,trend,True,NaN


In [26]:
labeled_df_setup.meta_df.index.name

'variable'

Now, we've set this up, we can also save these configurations to load them in directly in the future

In [27]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [28]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [10]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,[trend],True,NaN


In [11]:
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [12]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

Index(['year', 'search_conducted_false', 'search_conducted_true',
       'search_conducted_rate', 'contraband_found_false',
       'contraband_found_true', 'contraband_found_rate', 'hit_false',
       'hit_true', 'hit_rate', 'num_stops'],
      dtype='object', name='variable')

In [13]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [14]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,group_feat,subgroup,trend_type,agg_trend
0,year,search_conducted_false,0.299408,state,AZ,pearson_corr,-0.043351
1,year,search_conducted_false,-0.165978,state,CO,pearson_corr,-0.043351
2,year,search_conducted_false,-0.052226,state,CT,pearson_corr,-0.043351
3,year,search_conducted_false,-0.084114,state,IL,pearson_corr,-0.043351
4,year,search_conducted_false,0.118854,state,MA,pearson_corr,-0.043351


Now we can use a list of objects and apply multiple trends

In [15]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trendquality,trend_type,agg_trend,agg_trendquality
185,hit_rate,driver_race,state,CO,"[Black, Hispanic, Asian, White, Other]",NaN,rank_trend,"[Hispanic, Asian, Other, Black, White]",NaN
855,search_conducted_true,num_stops,no_reforms,1,29.9723,0.855259,lin_reg,34.8073,0.876247
101,contraband_found_true,driver_race,state,TX,"[Other, Asian, Black, Hispanic, White]",NaN,rank_trend,"[Asian, Other, Hispanic, Black, White]",NaN
1538,hit_false,num_stops,state,VT,NaN,NaN,lin_reg,NaN,NaN
747,search_conducted_true,hit_false,state,IL,0.793098,0.991824,lin_reg,NaN,NaN
224,num_stops,driver_race,medical,0,"[Other, Asian, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN
974,search_conducted_rate,hit_true,state,NC,NaN,NaN,lin_reg,NaN,NaN
1321,contraband_found_true,hit_rate,driver_race,Hispanic,NaN,NaN,lin_reg,NaN,NaN
1423,contraband_found_rate,hit_rate,state,RI,NaN,NaN,lin_reg,NaN,NaN
619,search_conducted_false,num_stops,driver_race,Asian,1.02007,0.999950,lin_reg,1.0224,0.999923


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [16]:
pd.unique(labeled_df.result_df['trend_type'])

array(['rank_trend', 'lin_reg'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [17]:
labeled_df.trend_list

In [18]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance()

In [19]:
labeled_df.result_df

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trendquality,trend_type,agg_trend,agg_trendquality,distance
0,search_conducted_false,driver_race,state,AZ,"[Asian, Black, Other, Hispanic, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.40
1,search_conducted_false,driver_race,state,CO,"[Other, Asian, Black, Hispanic, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.00
2,search_conducted_false,driver_race,state,CT,"[Other, Asian, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.20
3,search_conducted_false,driver_race,state,IL,"[Other, Asian, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.20
4,search_conducted_false,driver_race,state,MA,"[Other, Asian, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.20
5,search_conducted_false,driver_race,state,MD,"[Asian, Other, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.40
6,search_conducted_false,driver_race,state,NC,"[Asian, Other, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.40
7,search_conducted_false,driver_race,state,RI,"[Other, Asian, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.20
8,search_conducted_false,driver_race,state,SC,"[Other, Hispanic, Black, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.26
9,search_conducted_false,driver_race,state,TX,"[Other, Asian, Black, Hispanic, White]",NaN,rank_trend,"[Other, Asian, Black, Hispanic, White]",NaN,0.00


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [20]:
labeled_df.trend_list[0].trend_precompute

{'rank_trend_agg_trend_contraband_found_false_driver_race': driver_race
 Asian         6228.175532
 Black        41109.611354
 Hispanic     43598.262222
 Other         3885.717949
 White       177798.633621
 Name: contraband_found_false, dtype: float64,
 'rank_trend_agg_trend_contraband_found_false_state': state
 AZ     34835.937500
 CO     36792.271186
 CT     13741.260870
 IL     38969.150000
 MA     33690.660000
 MD     19395.280702
 NC     60045.880503
 RI      5031.521277
 SC     87343.697917
 TX    226487.010000
 VT      5476.440000
 WA     78856.887500
 WI     13404.328358
 Name: contraband_found_false, dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_driver_race': driver_race
 Asian       0.003160
 Black       0.012473
 Hispanic    0.008829
 Other       0.008818
 White       0.005542
 Name: contraband_found_rate, dtype: float64,
 'rank_trend_agg_trend_contraband_found_rate_state': state
 AZ    0.016694
 CO    0.002454
 CT    0.006426
 IL    0.007674
 MA    0.008515
